# Malicious PDF detection

The idea is to create some valid PDF files that, when opened, would "phone home" so to speak (in other words, trigger a connection to some URL that we specify).

These techniques can be benign, like checking whether someone has opened the document. They can also be used for malicious purposes, like exploiting the receiver's PDF rendering engine, and sending back a reverse shell. This proof-of-concept (PoC) is therefore a mere indicator of exploitability.

Once generated, a random forest classifier is trained to recognise various patterns in the malicious PDF files for later detection.

#### In this example, the data set has already been pre-generated.

## Step 1 - Generating malicious PDFs

We will use [https://github.com/jonaslejon/malicious-pdf](https://github.com/jonaslejon/malicious-pdf) (Batch A) and [https://blog.didierstevens.com/2011/05/25](https://blog.didierstevens.com/2011/05/25) (Batch B).

Currently they are labelled batch-X/YY.pdf? where X is `a` or `b`, YY is a digit placeholder, and ? is a possible additional file extension (such as `file.pdf.vir` where `.vir` means "virus" and prevents it from being opened).

## Step 2 - Preparing the data for training

Here we import all the necessary modules, preprocess and split the data, and define the necessary algorithms to use for training.

In [1]:
import glob
from pandas import read_csv, DataFrame
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from feature_extraction import feature_extraction

# Load prepared data set
df = read_csv('pdfdataset_n.csv')

# Choose appropriate fields
X = df.iloc[:, 0: 21]
y = df.iloc[:, 21]

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3)

# Various algorithms to try
algs = (
 ( 'Random Forest', RandomForestClassifier ),
 ( 'Support Vector Machine', SVC ),
 ( 'Linear Support Vector', LinearSVC ),
 ( 'Multilayer Perceptron', MLPClassifier ),
 ( 'K-Nearest Neighbours', KNeighborsClassifier )
)

# Answers for which PDFs are malicious
with open('solutions.txt') as f:
    solutions = {(a:=i.split())[0]:a[1] for i in f}

We may need to print each individual test, so let us define a control variable `DEBUG` that can easily be toggled on and off.

In [2]:
DEBUG = False

## Step 3, 4, 5 - Train, Test, Repeat (with different algorithms)

We will use the previously defined machine learning algorithms, printing each one out, training them, testing them with the pre-prepared data and then against our own PDF batches.

In [3]:
for name, method in algs:
    print('\n' + name + '\n' + '-'*len(name))
    model = method()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acs = round(accuracy_score(y_test, y_pred) * 100, 2)
    print(f'Training accuracy: {acs}%')

    results = {}
    correct = []
    for i in glob.iglob('**/*.pdf*', recursive=True):
        try:
            features = DataFrame(feature_extraction(i))
            result = model.predict(features)[0][0].upper()
            results[result] = results.get(result, 0) + 1
            correct.append(result == solutions[i.split('.')[0]])
            if DEBUG:
                print(f'{result}: {i}') # Key: Y is malicious, N is clean.
        except Exception:
            print('Error:', i)

    if DEBUG:
        for k, v in results.items():
            print(f'{v} x {k}')

    accuracy = correct.count(True) / len(correct) * 100
    print(f'Final accuracy: {round(accuracy, 2)}%')


Random Forest
-------------
Training accuracy: 99.93%
Final accuracy: 81.25%

Support Vector Machine
----------------------
Training accuracy: 99.38%
Final accuracy: 78.12%

Linear Support Vector
---------------------
Training accuracy: 99.37%
Final accuracy: 71.88%

Multilayer Perceptron
---------------------
Training accuracy: 99.72%
Final accuracy: 87.5%

K-Nearest Neighbours
--------------------
Training accuracy: 99.67%
Final accuracy: 81.25%
